## Getting Dataset

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

The dataset is present as a zip file. So we need to unzip the file. At this step we need to give the **source path** and the **destination path**. The file will be unzipped and stored at the destination path.

In [ ]:
#hlpr = !unzip "/content/drive/MyDrive/Classifier_Dataset.zip" -d "/content/"

# Importing Important Libraries

In [5]:
# this is going to help in data processing
from keras.preprocessing.image import ImageDataGenerator

In [6]:
#these are going to help in the model building.
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

# Setting up Variables

In [7]:
# dimensions of our images.
img_width, img_height = 150,150

#setting up the directories
train_data_dir = '/kaggle/input/classifier-dataset/Classifier_Dataset/Train'
validation_data_dir = '/kaggle/input/classifier-dataset/Classifier_Dataset/Validate'

#setting up the batchsizes.
nb_train_samples = 8200
nb_validation_samples = 1800
epochs = 50
batch_size = 16

In [8]:
input_shape = (img_width, img_height, 3)
input_shape

(150, 150, 3)

# Building the CNN model

In [9]:
#adding more conv2D layer
#previous two model combination

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))



model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))



model.add(Flatten())

model.add(Dense(700, activation='relu')) #add
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(1, activation='sigmoid'))


#configuring the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#to print a summary representation of your model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 148, 148, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 72, 72, 64)       2

# Data Augmentation

In [13]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=30,
    vertical_flip=True,
)

#this generates batches of augment data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 8200 images belonging to 2 classes.


In [14]:
# this is the augmentation configuration we will use for validating
val_datagen = ImageDataGenerator(rescale=1/255)

#this generates batches of augment data for validating
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 1800 images belonging to 2 classes.


# Training the model (Forward + Back Propagation)


In [15]:
model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/50


2023-07-15 10:42:10.268968: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


512/512 [==============================] - 156s 282ms/step - loss: 0.7197 - accuracy: 0.6464 - val_loss: 0.7006 - val_accuracy: 0.6233
Epoch 2/50
512/512 [==============================] - 98s 191ms/step - loss: 0.5602 - accuracy: 0.7198 - val_loss: 0.5316 - val_accuracy: 0.7500
Epoch 3/50
512/512 [==============================] - 97s 190ms/step - loss: 0.5143 - accuracy: 0.7499 - val_loss: 0.5881 - val_accuracy: 0.6920
Epoch 4/50
512/512 [==============================] - 96s 187ms/step - loss: 0.4968 - accuracy: 0.7650 - val_loss: 0.5713 - val_accuracy: 0.7165
Epoch 5/50
512/512 [==============================] - 96s 188ms/step - loss: 0.4903 - accuracy: 0.7625 - val_loss: 0.5301 - val_accuracy: 0.7400
Epoch 6/50
512/512 [==============================] - 95s 186ms/step - loss: 0.4838 - accuracy: 0.7691 - val_loss: 0.5412 - val_accuracy: 0.7461
Epoch 7/50
512/512 [==============================] - 93s 182ms/step - loss: 0.4748 - accuracy: 0.7705 - val_loss: 0.5829 - val_accuracy: 0.

# Saving Model

In [16]:
#to save the weights in the model as a HDFS file

model.save_weights('b_model_weight.h5')

In [17]:
#to save the architecture of the model as a json file

with open('b_model_architecture.json','w') as f:
    f.write(model.to_json())

# Restoring the Model

In [18]:
from keras.models import load_model
from keras.models import model_from_json 

In [19]:
# Model reconstruction from JSON file
with open('/kaggle/input/model-restore/b_model_architecture.json', 'r') as f:
    model1 = model_from_json(f.read())

In [20]:
# Load weights into the new model
model1.load_weights('/kaggle/input/model-restore/b_model_weight.h5')

# Testing the model

In [63]:
import numpy as np
#from keras.preprocessing import image
#from tensorflow.keras.preprocessing import image
import keras.utils as image

path = '/kaggle/input/test-model/new/new/download (1).jpg'
test_image=image.load_img(path,target_size=(150,150))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

result=model.predict(test_image)
print(result)

1/1 [==============================] - 0s 20ms/step
[[1.]]


In [65]:
if result[0][0]>=1.0:
    prediction='SoftDrink'
else:
    prediction='Pizza'
print("You got a "+ prediction + " enjoy!!")

You got a SoftDrink enjoy!!
